In [1]:
import os
os.chdir(r"C:\Users\macal\OneDrive\Documents\OMSCS\strategy_evaluation")
import pandas as pd
import numpy as np
import datetime
os.chdir(r"..")
from util import get_data
# from strategy_evaluation.indicators import BB_Pct, MACD, Generic_Cross, Stochastic_Oscillators
from strategy_evaluation.indicators import BB_Pct_Digital, MACD_Digital, Stochastic_Oscillators_Digital, Cross_Digital
from strategy_evaluation.marketsimcode import compute_portvals, create_holdings
os.chdir(r"strategy_evaluation")
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import math
from itertools import product
import numpy as np

In [2]:
def create_prices(dates, symbols):
    prices = get_data(dates=dates, symbols=symbols)
    prices = prices.dropna(subset=["SPY"])
    prices = prices[symbols]
    prices = prices.fillna(method="ffill").fillna(method="bfill")
    prices["Cash"] = 1.0
    return prices

def most_common_signal(row):
    # Count the occurrences of each value in the row, ignoring zeros
    counts = row[row != 0].value_counts()
    if counts.empty:
        return 0  # if all values are zero, return 0
    if len(counts) > 1 and counts.iloc[0] == counts.iloc[1] and counts.index[0] == -counts.index[1]:
        return 0  # if there's a tie between -1 and 1, return 0
    return counts.idxmax()  # return the most common non-zero value

def create_signals(indicator_signals,symbol):
    # indicator_signals["signal"] = (indicator_signals!=0).sum(axis=1)
    signal = indicator_signals.apply(lambda signals: most_common_signal(signals), axis = 1)
    return signal.to_frame().rename(columns={0:symbol})

def buy_action(share_count:int):
    return 1000-share_count
def sell_action(share_count:int):
    return -1000 - share_count

def create_trades(prices, signals, commission=0, impact=0):
    symbol = prices.columns[0]
    trades = prices.copy()
    trades[symbol]=0
    trades["Cash"]=0
    for i,r in signals.iterrows():
        total_shares = trades.loc[:i,symbol].sum()
        if r[symbol] == 1:
            action = buy_action(total_shares)
        if r[symbol] == -1:
            action = sell_action(total_shares)
        if r[symbol] == 0:
            action = 0
        if i==signals.index.min():
            action = buy_action(total_shares)
        trades.loc[i,symbol]=action
        trades.loc[i,"Cash"]=-action*prices.loc[i,symbol]
    return trades

def get_port_vals(prices, sv, indicators,symbol):
    signals = create_signals(indicators, symbol)
    trades = create_trades(prices, signals)
    holdings = trades.copy()
    holdings.iloc[0,1] += sv
    holdings = holdings.cumsum()
    values = prices * holdings
    port_vals = values.sum(axis=1)
    return port_vals
    
def get_cr(port_val):
    """
    This function will return the cumulative return of a profile
    According to the lectures:
    Cumulative Return = (port_val[-1]/port_val[0]) - 1

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A pandas DataFrame object that represents the cumulative return values of the portfolio
    :rtype: pd.DataFrame
    """
    cr = (port_val.iloc[-1] / port_val.iloc[0]) - 1
    return cr

def get_daily_rets(port_val):
    """
    This function will return the average daily return of a profile
    According to the lectures:
    daily_rets = (df[1:]/df[:-1].values)-1
    Or in other words, today's value / yesterday's value

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A pandas DataFrame object that represents the daily return values of the portfolio
    :rtype: pd.DataFrame
    """
    daily_rets = (port_val / port_val.shift(1)) - 1
    return daily_rets

def get_adr(port_val):
    """
    This function will return the average daily return of a profile
    According to the lectures:
    Average Daily Return = daily_rets.mean()

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A float that is equal to the average values of the portfolio
    :rtype: float
    """
    daily_rets = get_daily_rets(port_val)
    adr = daily_rets.mean()
    return adr

def get_sddr(port_val):
    """
    This function will return the Standard Deviation of Daily Return of a profile
    According to the lectures:
    Standard Deviation of Daily Return = daily_rets.std()
    Note: We need sample standard deviation (Thank you Andrew Rife)

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A float that is equal to the standard deviation values of the portfolio
    :rtype: float
    """
    daily_rets = get_daily_rets(port_val)
    sddr = np.std(daily_rets,ddof=1)
    return sddr

def get_sr(port_val, risk_free_rate = 0):
    """
    This function will return the Sharpe Ratio of a profile

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :param risk_free_rate: A float that represents the risk-free rate used in the Sharpe Ratio Calculation
    :type risk_free_rate: float
    :return: The Sharpe Ratio of the portfolio provided
    :rtype: float
    """
    sr = ((get_adr(port_val)-risk_free_rate) / get_sddr(port_val))*math.sqrt(252)
    return sr

In [3]:
sd = datetime.datetime(2008,1,1)
ed = datetime.datetime(2009,12,31)
symbol = "JPM"
prices = create_prices(
    dates = pd.date_range(sd, ed),
    symbols=[symbol]
)
starting_value = 100_000

In [4]:
window = 20
fast = 12
slow = 26
signal_window = 9
k = 14
d = 3
overbought_thresh = 0.8
oversold_thresh = 0.2
short = 50
long = 200

bb_prices = BB_Pct_Digital(prices = prices.drop(columns="Cash"), window = window).rename(columns={symbol:"BB_Pct"}).sort_index()
# macd_prices = MACD_Digital(prices = prices.drop(columns="Cash"), fast = fast, slow = slow, signal_window=signal_window).rename(columns={symbol:"MACD"}).sort_index()
stocs_prices = Stochastic_Oscillators_Digital(prices = prices.drop(columns="Cash"), k = k, d = d, overbought_thresh=overbought_thresh, oversold_thresh=oversold_thresh).rename(columns={symbol:"Stochastic_Oscillators"}).sort_index()
cross_prices = Cross_Digital(prices = prices.drop(columns="Cash"), short = short, long = long).rename(columns={symbol:"Cross"}).sort_index()
# indicators = pd.concat([bb_prices,macd_prices,stocs_prices,cross_prices],axis = 1)


window: an integer value between 10 to 50 - Try 21  
fast: an integer value between 5 to 20 - Try 11  
slow: an integer value between 20 to 30 - Try 23  
signal_window: an integer value between 5 to 15 - Try 9  
k: an integer value between 5 to 21 - Try 14  
d: an integer value between 3 to 5 - Try 3  
overbought_thresh: a float value between 0.7 to 0.9 - Try 0.76  
oversold_thresh: an integer value between 0.1 to 0.3 - Try 0.21  
short: an integer value between 30 to 60 - Try 52  
long: an integer value between 100 to 300 - Try 205  

Initial: [20, 12, 26, 9, 14, 3, 0.8, 0.2, 50, 200]

Nelder-Mead Optimal parameters: [ 21.18171415  11.50643598  23.27708297   8.90634184  14.52437511
   2.98343726   0.76660534   0.21000367  51.81315595 205.14106227]  

SLSQP Optimal parameters: parameters: [ 20.   12.   26.    9.   14.    3.    0.8   0.2  50.  200. ]


COBYLA Optimal parameters: [ 20.8574958   11.83947381  25.78698768   9.86245646  13.84988208
   4.43553632   0.64797776   0.22465324  51.24688173 199.56033737]

In [9]:
#January 1, 2008, to December 31, 2009. 
sd = datetime.datetime(2008,1,1)
ed = datetime.datetime(2009,12,31)
sv = 100000
dates = pd.date_range(sd, ed)
symbol = "JPM"
symbols = [symbol]
prices = get_data(symbols = symbols , dates = dates)
prices = prices.dropna(subset=["SPY"])[symbols]
prices = prices.fillna(method="ffill").fillna(method="bfill").sort_index()
d = 3
fast = 5
k = 5
long = 100
overbought_thresh = 0.75
oversold_thresh = 0.2
short = 30
signal_window = 5
slow = 20
window = 10

bb_prices = BB_Pct_Digital(prices = prices, window = window).rename(columns={symbol:"BB_Pct"}).sort_index()
macd_prices = MACD_Digital(prices = prices, fast = fast, slow = slow, signal_window=signal_window).rename(columns={symbol:"MACD"}).sort_index()
stocs_prices = Stochastic_Oscillators_Digital(prices = prices, k = k, d = d, overbought_thresh=overbought_thresh, oversold_thresh=oversold_thresh).rename(columns={symbol:"Stochastic_Oscillators"}).sort_index()
cross_prices = Cross_Digital(prices = prices, short = short, long = long).rename(columns={symbol:"Cross"}).sort_index()
def convert_digital_signal(signal):
    if signal ==  1: return "BUY"
    if signal == -1: return "SELL"
    return "HOLD"

def buy_action(share_count:int):
    return 1000-share_count
def sell_action(share_count:int):
    return -1000 - share_count

def most_common_signal(row):
    # Count the occurrences of each value in the row, ignoring zeros
    counts = row[row != 0].value_counts()
    if counts.empty:
        return 0  # if all values are zero, return 0
    if len(counts) > 1 and counts.iloc[0] == counts.iloc[1] and counts.index[0] == -counts.index[1]:
        return 0  # if there's a tie between -1 and 1, return 0
    return counts.idxmax()  # return the most common non-zero value


def create_TOS(dates,symbol):
    symbols = [symbol]
    
    prices = create_prices(dates=dates,symbols=symbols)
    
    trades = pd.DataFrame(0.0, index=prices.index, columns=prices.columns)
    for i, (date,values) in enumerate(trades.iloc[:-1].iterrows()):
        total_shares = trades.loc[:date,symbols].sum()[0]
        is_tomorrow_more = (prices.iloc[i+1]>prices.iloc[i])[symbol]
        if is_tomorrow_more: 
            action = buy_action(total_shares)
        else:
            action = sell_action(total_shares)
        trades.loc[date,symbols]=action
        trades.loc[date,"Cash"]=-action*prices.loc[date,symbol]
    return trades

def create_benchmark(dates,symbol):
    symbols = [symbol]
    
    prices = create_prices(dates=dates,symbols=symbols)
    
    benchmark = pd.DataFrame(0.0, index=prices.index, columns=prices.columns)
    benchmark.iloc[0][symbols] = 1000
    benchmark.iloc[0]["Cash"] = prices.iloc[0][symbols]*-1000
    return benchmark

def get_cr(port_val):
    """
    This function will return the cumulative return of a profile
    According to the lectures:
    Cumulative Return = (port_val[-1]/port_val[0]) - 1

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A pandas DataFrame object that represents the cumulative return values of the portfolio
    :rtype: pd.DataFrame
    """
    cr = (port_val.iloc[-1] / port_val.iloc[0]) - 1
    return cr


def get_daily_rets(port_val):
    """
    This function will return the average daily return of a profile
    According to the lectures:
    daily_rets = (df[1:]/df[:-1].values)-1
    Or in other words, today's value / yesterday's value

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A pandas DataFrame object that represents the daily return values of the portfolio
    :rtype: pd.DataFrame
    """
    daily_rets = (port_val / port_val.shift(1)) - 1
    return daily_rets


def get_adr(port_val):
    """
    This function will return the average daily return of a profile
    According to the lectures:
    Average Daily Return = daily_rets.mean()

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A float that is equal to the average values of the portfolio
    :rtype: float
    """
    daily_rets = get_daily_rets(port_val)
    adr = daily_rets.mean()
    return adr


def get_sddr(port_val):
    """
    This function will return the Standard Deviation of Daily Return of a profile
    According to the lectures:
    Standard Deviation of Daily Return = daily_rets.std()
    Note: We need sample standard deviation (Thank you Andrew Rife)

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :return: A float that is equal to the standard deviation values of the portfolio
    :rtype: float
    """
    daily_rets = get_daily_rets(port_val)
    sddr = np.std(daily_rets,ddof=1)
    return sddr


def get_sr(port_val, risk_free_rate = 0):
    """
    This function will return the Sharpe Ratio of a profile

    :param port_val: A pandas DataFrame object that contains the portfolio value
    :type port_val: pd.DataFrame
    :param risk_free_rate: A float that represents the risk-free rate used in the Sharpe Ratio Calculation
    :type risk_free_rate: float
    :return: The Sharpe Ratio of the portfolio provided
    :rtype: float
    """
    sr = ((get_adr(port_val)-risk_free_rate) / get_sddr(port_val))*math.sqrt(252)
    return sr

def create_chart(port_vals,benchmark_vals, tos_vals,symbol):
    fig, ax = plt.subplots()
    benchmark_vals.plot(ax = ax, color = "purple", label="Benchmark Strategy")
    tos_vals.plot(ax = ax, color = "red", label="Theoretically Optimal Strategy")
    port_vals.plot(ax = ax, color = "green", label="Manual Strategy")
    ax.grid(visible=True,linestyle=':')
    ax.legend()
    ax.set_title(label=f"ManualStrategy vs TheoreticallyOptimalStrategy vs Benchmark on {symbol} symbol")
    ax.set_xlabel(xlabel="Date")
    ax.set_ylabel(ylabel="Normalized Value")
    plt.savefig("p6_tos_chart.png")
    
def create_table(port_vals,bechmark_vals):
    port_vals_data = [
        get_cr(port_vals),
        get_adr(port_vals),
        get_sddr(port_vals),
        get_sr(port_vals)
    ]
    benchmark_data = [
        get_cr(bechmark_vals),
        get_adr(bechmark_vals),
        get_sddr(bechmark_vals),
        get_sr(bechmark_vals)
    ]
    port_stats = pd.DataFrame(data={"Portfolio":port_vals_data,
                                    "Benchmark":benchmark_data},
                              index=["cum_ret","avg_dr","ssdr","sharpe_ratio"])
    port_stats = np.round(a=port_stats,decimals=6)


In [11]:
signals = create_signals(bb_prices, symbol)
trades = create_trades(prices, signals)
prices["Cash"]=1
holdings = trades.copy()
holdings.iloc[0,1] += sv
holdings = holdings.cumsum()
values = prices * holdings
port_vals = values.sum(axis=1)
get_sr(port_vals/port_vals.iloc[0])

1.1722876132719364

In [35]:
def objective(params):
    window, fast, slow, signal_window, k, d, overbought_thresh, oversold_thresh, short, long = params
    # Round integer parameters
    window = int(round(window))
    fast = int(round(fast))
    slow = int(round(slow))
    signal_window = int(round(signal_window))
    k = int(round(k))
    d = int(round(d))
    short = int(round(short))
    long = int(round(long))
    
    # Calculate indicators
    bb = BB_Pct_Digital(prices.drop(columns=["Cash"]), int(window))
    macd = MACD_Digital(prices.drop(columns=["Cash"]), int(fast), int(slow), int(signal_window))
    stoch = Stochastic_Oscillators_Digital(prices.drop(columns=["Cash"]), int(k), int(d), overbought_thresh, oversold_thresh)
    cross = Cross_Digital(prices.drop(columns=["Cash"]), int(short), int(long))
    
    indicators = pd.concat([bb, cross,stoch, macd],axis=1)
    signals = create_signals(indicators, symbol)
    trades = create_trades(prices, signals)
    holdings = trades.copy()
    holdings.iloc[0,1] += sv
    holdings = holdings.cumsum()
    values = prices * holdings
    port_vals = values.sum(axis=1)
    
    # Get the Sharpe Ratio
    sharpe_ratio = get_sr(port_vals/port_vals.iloc[0])
    return -sharpe_ratio

# Bounds for the parameters
bounds = [(10, 50), (5, 20), (20, 30), (5, 15), (5, 21), (3, 5),
          (0.7, 0.9), (0.1, 0.3), (30, 60), (100, 300)]

# Initial guess (midpoint of bounds)
initial_guess = initial_guess = [20, 12, 26, 9, 14, 3, 0.8, 0.2, 50, 200]

# Perform minimization
result = minimize(objective, initial_guess, bounds=bounds, method='Powell')

# Print the optimal parameters
print("Optimal parameters:", result.x)
print("Minimum Sharpe Ratio:", result.fun)

Optimal parameters: [2.06286771e+01 1.20000000e+01 2.50172209e+01 9.62867712e+00
 1.05983935e+02 3.00000000e+00 3.53752855e-01 2.00000000e-01
 5.06286771e+01 2.01222400e+02]
Minimum Sharpe Ratio: -0.9459260953498969
